# In-Class Coding Lab: Understanding The Foundations of Web APIs

### Overview

This lab covers the foundations of what is necessary to properly use consume HTTP web service API's with Python . Here's what we will cover.

1. Understading requests and responses
1. Proper error handling
1. Parameter handling
1. Refactoring as a function


In [ ]:
# Run this to make sure you have the pre-requisites!
!pip install -q requests

## Part 1: Understanding Requests and responses

In this part we learn about the Python requests module. http://docs.python-requests.org/en/master/user/quickstart/ 

This module makes it easy to write code to send HTTP requests over the internet and handle the responses. It will be the cornerstone of our API consumption in this course. While there are other modules which accomplish the same thing, `requests` is the most straightforward and easiest to use.

We'll begin by importing the modules we will need. We do this here so we won't need to include these lines in the other code we write in this lab.

In [ ]:
# start by importing the modules we will need
import requests
import json 

### The request 

As you learned in class and your assigned readings, the HTTP protocol has **verbs** which consititue the type of request you will send to the remote resource, or **url**. Based on the url and request type, you will get a **response**.

The following line of code makes a **get** request (that's the HTTP verb) to Google's Geocoding API service. This service attempts to convert the address (in this case `Syracuse University`) into a set of coordinates global coordinates (Latitude and Longitude), so that location can be plotted on a map.


In [ ]:
url = 'https://nominatim.openstreetmap.org/search?q=Hinds+Hall+Syracuse+University&format=json'
response = requests.get(url)

### The response 

The `get()` method returns a `Response` object variable. I called it `response` in this example but it could be called anything. 

The HTTP response consists of a *status code* and *body*. The status code lets you know if the request worked, while the body of the response contains the actual data. 


In [ ]:
response.ok # did the request work?

In [ ]:
response.text  # what's in the body of the response, as a raw string

### De-Serializing JSON Text into Python object variables 

In the case of **web site url's** the response body is **HTML**. This should be rendered in a web browser. But we're dealing with Web Service API's so...

In the case of **web API url's** the response body could be in a variety of formats from **plain text**, to **XML** or **JSON**. In this course we will only focus on JSON format because as we've seen these translate easily into Python object variables.

Let's convert the response to a Python object variable.

In [ ]:
geodata = response.json()  # try to decode the response from JSON format
geodata                    # this is now a Python object variable

With our Python object, we can now walk the list of dictionary to retrieve the latitude and longitude


In [ ]:
lat = geodata[0]['lat']
lon =geodata[0]['lon']
print(lat, lon)

In the code above we "walked" the Python list of dictionary to get to the location

- `geodata` is a list
- `geodata[0]` is the first item in that list, a dictionary
- `geodata[0]['lat']` is a dictionary key which represents the latitude 
- `geodata[0]['lon']` is a dictionary key which represents the longitude

It should be noted that this process will vary for each API you call, so its important to get accustomed to performing this task. You'll be doing it quite often. 

One final thing to address. What is the type of `lat` and `lon`?

In [ ]:
type(lat), type(lon)

Bummer they are strings. we want them to be floats so we will need to parse the strings with the `float()` function:

In [ ]:
lat = float(geodata[0]['lat'])
lon = float(geodata[0]['lon'])
print("Latitude: %f, Longitude: %f" % (lat, lon))

## What did we just do?

At this stage, the process for calling a WebAPI in JSON format using Python is the same, regardless of the API.


1. Use `requests.get(url)` to make an HTTP GET request to the `url`.
2. Assuming the `response.ok` we can `response.json()` to de-serialize the JSON into a Python object.
3. We then extract the information we need using the typical Python `list` and `dict` methods.


### 1.1 You Code

This url calls the [GovTrack API](https://www.govtrack.us/), and retrieves information regarding the current President of the United States.

[https://www.govtrack.us/api/v2/role?current=true&role_type=president](https://www.govtrack.us/api/v2/role?current=true&role_type=president)

 1. Use `requests.get()` to retrieve the contents of the API at this url.
 2. Use `response.json()` to de-serialize the the response JSON text to a Python object.
 3. Find and print the `"name` of the current president by locating it within the Python object.
 
**HINT**: to figure that out, click on the URL and view the content in your broswer.


In [ ]:
# TODO Write code here
url = 'https://www.govtrack.us/api/v2/role?current=true&role_type=president'

## Part 2: Parameter Handling

In the example above we hard-coded `current=true` and `role_type=president` into the request:

`url = 'https://www.govtrack.us/api/v2/role?current=true&role_type=president'`

Likewise in the open stret map example we hard coded in the `Hinds Hall Syracuse University` part:

`url = 'https://nominatim.openstreetmap.org/search?q=Hinds+Hall+Syracuse+University&format=json'` 

A better way to write this code is to allow for the **input** of any location and supply that to the service. To make this work we need to send parameters into the request as a dictionary. **Parameters** end up being built into a **Query String** on the url which serve as the **inputs into the API Request**. 

This way we can geolocate any address!

You'll notice that on the url, we are passing **key-value pairs** the key is `q` and the value is `Hinds+Hall+Syracuse+University`. The other key is `format` and the value is `json`. Hey, Python dictionaries are also key-value pairs so:

In [ ]:
url = 'https://nominatim.openstreetmap.org/search'  # base URL without paramters after the "?"
search = 'Hinds Hall Syracuse University'
options = { 'q' : search, 'format' : 'json'}
response = requests.get(url, params = options)  # This builds the url
print(f"Requested URL: {response.url}") # print the built url
geodata = response.json()
coords = { 'lat' : float(geodata[0]['lat']), 'lng' : float(geodata[0]['lon']) }
print("Search for:", search)
print("Coordinates:", coords)
print(f"{search} is located at ({coords['lat']},{coords['lng']})")

### Looking up any address

RECALL: For `requests.get(url, params = options)` the part that says `params = options` is called a **named argument**, which is Python's way of specifying an optional function argument.

With our parameter now outside the url, we can easily re-write this code to work for any location! Go ahead and execute the code and input `Queens, NY`. This will retrieve the coordinates `(40.728224,-73.794852)`

In [ ]:
url = 'https://nominatim.openstreetmap.org/search'  # base URL without paramters after the "?"
search = input("Enter a loacation to Geocode: ")
options = { 'q' : search, 'format' : 'json'}
response = requests.get(url, params = options)            
geodata = response.json()
coords = { 'lat' : float(geodata[0]['lat']), 'lng' : float(geodata[0]['lon']) }
print("Search for:", search)
print("Coordinates:", coords)
print(f"{search} is located at ({coords['lat']},{coords['lng']})")

## This is so useful, it should be a function!

One thing you'll come to realize quickly is that your API calls should be wrapped in functions. This promotes **readability** and **code re-use**. For example:

In [ ]:
def get_coordinates(search):
    url = 'https://nominatim.openstreetmap.org/search'  # base URL without paramters after the "?"
    options = { 'q' : search, 'format' : 'json'}
    response = requests.get(url, params = options)            
    geodata = response.json()
    coords = { 'lat' : float(geodata[0]['lat']), 'lng' : float(geodata[0]['lon']) }
    return coords

# main program here:
location = input("Enter a location: ")
coords = get_coordinates(location)
print(f"{search} is located at ({coords['lat']},{coords['lng']})")


### 1.2 You Code: Debug

Get this code working! 

The [GovTrack API](https://www.govtrack.us/), allows you to retieve information about people in Government with 4 different role types: `senator, representative, president, vicepresident`  for example, when you add the `role_type=president` to the request URL you get the US president, whhen you add `role_type=senator` you get back US senators.

This code should present a drop down of roles. Upon selected the API is called for that role and then for each object in that role we print the `['person']['name']` as before. 


**HINT**: If you are getting errors, click on the response URL to see the API output.

In [ ]:
from ipywidgets import interact

roles = ['senator', 'representative', 'president', 'vicepresident' ]
@interact(role_type=roles)
def main(role_type):
    url = 'https://www.govtrack.us/api/v2/role' 
    params =  { 'current' : 'true', 'role_type' : "?" }
    response = requests.get(url, params = params)
    print(f"Requested URL: {response.url}")
    data = response.json
    for item in data['objects']:
        print(f"- persons name")

## Other request methods

Not every API we call uses the `get()` method. Some use `post()` because the amount of data you provide it too large to place on the url.  The `HTTP POST` method sends input data within the body of the request. It does NOT appear on the URL.

An example of an API that uses this method is the  **Text-Processing.com** sentiment analysis service. http://text-processing.com/docs/sentiment.html This service will detect the sentiment or mood of text. You give the service some text, and it tells you whether that text is positive, negative or neutral.  The JSON response has a key called `label` which provides the sentiment.

Examples:

In [ ]:
# 'you suck' == 'negative'
url = 'http://text-processing.com/api/sentiment/'
payload = { 'text' : 'you suck'}
response = requests.post(url, data = payload)
sentiment = response.json()
sentiment

In [ ]:
# 'I love cheese' == 'positive'
url = 'http://text-processing.com/api/sentiment/'
payload = { 'text' : 'I love cheese'}
response = requests.post(url, data = payload)
sentiment = response.json()
sentiment

In the examples provided we used the `post()` method instead of the `get()` method. the `post()` method has a named argument `data` which takes a dictionary of data, in HTTP parlance this is referred to as the **payload**. The payload is a dictionary and for **text-processing.com** it required a key `text` which holds the text you would like to process for sentiment.


Here's an example of processing the sentiment of a Tweet:

In [ ]:
tweet = "Arnold Schwarzenegger isn't voluntarily leaving the Apprentice, he was fired by his bad (pathetic) ratings, not by me. Sad end to a great show"
url = 'http://text-processing.com/api/sentiment/'
payload = { 'text' : tweet }
response = requests.post(url, data = payload)
sentiment = response.json()
print("TWEET:", tweet)
print("SENTIMENT", sentiment['label'])

## Applications

Sentiment analysis is a useful tool for getting a sense of the mood of text. Any text can be analyzed and common applications are analyzing social media, blog comments, product reviews, and open-ended sections of surveys.

### 1.3 You Code

Use the above example to write a program which will input any text and print the sentiment using this API!

In [ ]:
#TODO write code here



## Troubleshooting

When you write code that depends on other people's code from around the Internet, there's a lot that can go wrong. Therefore we perscribe the following advice:

```
Assume anything that CAN go wrong WILL go wrong
```

Let's put this to the test with the following example where we call an API to get the [IP Address](https://en.wikipedia.org/wiki/IP_address) of the computer making the call.


### First Things First: Know Your Errors!

Above all, the #1 thing you should understand are the errors you get from Python and what they mean.

Case in point: This first example, which produces a `JSONDecodeError` on line 3.

In [ ]:
url = "http://myip.ist256.com"
response = requests.get(url)
data = response.json()
print(data)

This means the response back we get from `"http://myip.ist256.com"` cannot be decoded from JSON to a Python object. 

You might start looking there but you're making a HUGE assumption that the service `"http://myip.ist256.com"` is "working".

NEVER make this assumption!

KNOW whether or not its working!

There are a couple ways you can do this:

- print the `response.url` then click on it to see what happens.
- make `reqests` throw an error on unsuccessful HTTP response codes.

Let's do both:

 - we add `print(response.url)` to see the actual URL we are sending to the API.
 - we add `response.raise_for_status()` which throws an exception if the response is not `200/OK`. 


In [ ]:
url = "http://myip.ist256.com"
response = requests.get(url)
print(f"Generated Url: {response.url}")
response.raise_for_status()
data = response.json()
print(data)

We no longer have a `JSONDecodeError` We now see the REAL error here an `HTTPError` response 503.

According to the `HTTP` Protocol spec, error 5xx means it's the server's problem. No amount of code will fix that. We need a different url.

Let's try this instead: `https://whatismyipaddress.com/`

In [ ]:
url = "https://whatismyipaddress.com/"
response = requests.get(url)
print(f"Generated Url: {response.url}")
response.raise_for_status()
data = response.json()
print(data)

This no longer has an `HTTPError`, but now we are back to the `JSONDecodeError`. The response from the URL cannot be de-serialized from JSON text.

NOW you should check - if the output of the response isn't JSON, what is it?

There are two ways you can do this:

 - Print the `response.url` and click on it to see if the output is JSON.
 - print `response.text` which is the raw output from the response.
 
 
We already have the first, let's add the second.



In [ ]:
url = "https://whatismyipaddress.com/"
response = requests.get(url)
print(f"Generated Url: {response.url}")
response.raise_for_status()
print(f"RAW RESPONSE: {response.text}")
data = response.json()
print(data)

As You can see, the response is:

    Access Denied (BUA77). Contact support@whatismyipaddress.com

which is not at all what we expected. Again no amount of Python code will fix this, we need to call the right API, or change the URL of this API.

As a final step, let's try this service: `http://httpbin.org/ip`

In [ ]:
url = "https://httpbin.org/ip"
response = requests.get(url)
print(f"Generated Url: {response.url}")
response.raise_for_status()
print(f"RAW RESPONSE: {response.text}")
data = response.json()
print(data)

**Now that works!**

The first is the raw response, and the second is the Python object.

To demonstrate its a python object, let's extract the IP Address from the `origin` key.

The intermediate `print()` statements have been removed since the code now works.

In [ ]:
url = "https://httpbin.org/ip"
response = requests.get(url)
response.raise_for_status()
data = response.json()
print(f"MY IP ADDRESS: {data['origin']}")

##  Guidelines for Rewriting as a function

To make your code clear and easier to read, its a good idea to re-factor your working API call into a function. Here are the guidelines:

- DO NOT write the function until you get the code working. ALWAYS re-factor (rewrite) the WORKING code as a function.
- One API call per function. Don't do too much!
- Inputs into the API call such as query string parameters or `POST` body text should be function input parameters.
- The function should NOT return the entire response unless its required. Only return what is needed.
- use `response.raise_for_status()` to throw `HTTPError` exceptions. This way you will not be misled when there is a problem with the API and not your code.
- DO NOT handle errors in your function or account for contingencies. Error handling is the responsilbity of the function's caller.


### 1.4 You Code

Refactor the code in the cell above into a function `iplookup()`. call the function to demonsrate it works.


In [ ]:
# TODO Your Code Here


# Metacognition



### Rate your comfort level with this week's material so far.   

**1** ==> I don't understand this at all yet and need extra help. If you choose this please try to articulate that which you do not understand to the best of your ability in the questions and comments section below.  
**2** ==> I can do this with help or guidance from other people or resources. If you choose this level, please indicate HOW this person helped you in the questions and comments section below.   
**3** ==> I can do this on my own without any help.   
**4** ==> I can do this on my own and can explain/teach how to do it to others.

`--== Double-Click Here then Enter a Number 1 through 4 Below This Line ==--`  


###  Questions And Comments 

Record any questions or comments you have about this lab that you would like to discuss in your recitation. It is expected you will have questions if you did not complete the code sections correctly.  Learning how to articulate what you do not understand is an important skill of critical thinking. Write them down here so that you remember to ask them in your recitation. We expect you will take responsilbity for your learning and ask questions in class.

`--== Double-click Here then Enter Your Questions Below this Line ==--`    


In [ ]:
# run this code to turn in your work!
from coursetools.submission import Submission
Submission().submit()